In [37]:
import pandas as pd
import random
import duckdb 
from faker import Faker 

fake = Faker()

# number of chickens/eggs to generate
n = 1000

def generate_ids(num_chickens):
    identifiers = set() #ensure uniqueness in an empty set
    while len(identifiers) < num_chickens:
        identifier = random.randint(10000000, 99999999)
        identifiers.add(identifier)
    return list(identifiers)

chickens = generate_ids(n)
chicken_df = pd.DataFrame(chickens, columns=['egg_id'])

In [38]:
# because we need to create lineage, i'm creating 'generations', where generation 3 is the most recent set of chickens
# gen 3's parents are gen 2, and gen 2's parents are gen 1
# assuming hens give birth to a lot of chickens, so setting a 10-30-60 ratio across gens 1 thru 3

num_gen1 = int(n * 0.1)
num_gen2 = int(n * 0.3)
num_gen3 = n - (num_gen1 + num_gen2)

generation_list = [1] * num_gen1 + [2] * num_gen2 + [3] * num_gen3
random.shuffle(generation_list)
chicken_df['generation'] = generation_list

chicken_df

,egg_id,generation
0,21592066,3
1,90327043,1
2,68022276,3
3,74606596,3
4,31432712,2
...,...,...
995,48175098,3
996,93218811,3
997,18358269,3
998,57497598,3


In [39]:
# function to assign rooster or hen to each chicken
def assign_gender_with_split(chickens, gender_split, gender1, gender2):
    half = len(chickens) // 2
    chickens.loc[chickens.index[:half], 'gender'] = gender1
    chickens.loc[chickens.index[half:], 'gender'] = gender2
    return chickens

# ensure 50-50 split between gender within each generation
gen1_df = chicken_df[chicken_df['generation'] == 1].copy()
gen2_df = chicken_df[chicken_df['generation'] == 2].copy()
gen3_df = chicken_df[chicken_df['generation'] == 3].copy()

gen1_df = assign_gender_with_split(gen1_df, 0.5, 'Rooster', 'Hen')
gen2_df = assign_gender_with_split(gen2_df, 0.5, 'Rooster', 'Hen')
gen3_df = assign_gender_with_split(gen3_df, 0.5, 'Rooster', 'Hen')

chicken_df = pd.concat([gen1_df, gen2_df, gen3_df], ignore_index=True)
chicken_df

,egg_id,generation,gender
0,90327043,1,Rooster
1,25692215,1,Rooster
2,75991113,1,Rooster
3,35881049,1,Rooster
4,54519912,1,Rooster
...,...,...,...
995,48175098,3,Hen
996,93218811,3,Hen
997,18358269,3,Hen
998,57497598,3,Hen


In [40]:
# let's create a lookup table for the incubation hall

incubation_hall_location_ids = list(range(1, 31))
is_near_window = [True if i <= 10 else False for i in incubation_hall_location_ids]

data = {
    "incubation_hall_location_id": incubation_hall_location_ids,
    "is_near_window": is_near_window
}

incubation_hall = pd.DataFrame(data)


In [41]:
# technically this is making it possible for a mother hen to have multiple incubation locations, but i'm assuming that's fine
chicken_df['incubation_hall_location_id'] = [random.choice(incubation_hall_location_ids) if generation != 1 else None for generation in chicken_df['generation']]

# check to make sure that roughly 33% of chickens were incubated near a window
duckdb.query("""
with counts as (
    select incubation_hall_location_id, count(*) as count from chicken_df where generation != 1 group by 1 order by 1
)

, windowed_sums as (
select
    *,
    sum(count) over (order by incubation_hall_location_id rows between unbounded preceding and current row) as cumulative_count,
    sum(count) over (order by incubation_hall_location_id rows between unbounded preceding and unbounded following) as total_count
from counts
)

select
    *,
    cumulative_count::float / total_count::float as cumulative_percent
from windowed_sums
 """).to_df()


,incubation_hall_location_id,count,cumulative_count,total_count,cumulative_percent
0,1.0,27,27.0,900.0,0.030000
1,2.0,31,58.0,900.0,0.064444
2,3.0,24,82.0,900.0,0.091111
3,4.0,30,112.0,900.0,0.124444
4,5.0,18,130.0,900.0,0.144444
5,6.0,30,160.0,900.0,0.177778
6,7.0,29,189.0,900.0,0.210000
7,8.0,34,223.0,900.0,0.247778
8,9.0,39,262.0,900.0,0.291111
9,10.0,35,297.0,900.0,0.330000


In [42]:
# validate that the geneder split function works correctly
duckdb.query("select generation, gender, count(*) as count from chicken_df group by 1,2").to_df()

,generation,gender,count
0,1,Rooster,50
1,1,Hen,50
2,2,Rooster,150
3,2,Hen,150
4,3,Rooster,300
5,3,Hen,300


In [43]:
# using the faker library to generate full names for each chicken dependent on gender
chicken_df['name'] = [
    f"{fake.first_name_male()} {fake.last_name()}" if gender == 'Rooster'
    else f"{fake.first_name_female()} {fake.last_name()}"
    for gender in chicken_df['gender']
]

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name
0,90327043,1,Rooster,NaN,Mark Hernandez
1,25692215,1,Rooster,NaN,Victor Wells
2,75991113,1,Rooster,NaN,Jeffrey Walls
3,35881049,1,Rooster,NaN,Ryan Green
4,54519912,1,Rooster,NaN,Joseph Anderson
...,...,...,...,...,...
995,48175098,3,Hen,8.0,Michelle Chandler
996,93218811,3,Hen,18.0,Mary Pope
997,18358269,3,Hen,7.0,Rachel Berry
998,57497598,3,Hen,2.0,Mary George


In [44]:
# generate random feather colors for each chicken
def random_feather_color():
    colors = ['white', 'black', 'brown', 'red', 'gray', 'gold']
    return random.choice(colors)

chicken_df['feather_color'] = [random_feather_color() for i in range(len(chicken_df))]

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name,feather_color
0,90327043,1,Rooster,NaN,Mark Hernandez,white
1,25692215,1,Rooster,NaN,Victor Wells,gold
2,75991113,1,Rooster,NaN,Jeffrey Walls,gold
3,35881049,1,Rooster,NaN,Ryan Green,brown
4,54519912,1,Rooster,NaN,Joseph Anderson,black
...,...,...,...,...,...,...
995,48175098,3,Hen,8.0,Michelle Chandler,black
996,93218811,3,Hen,18.0,Mary Pope,gray
997,18358269,3,Hen,7.0,Rachel Berry,red
998,57497598,3,Hen,2.0,Mary George,black


In [45]:
# using a free API to get the billboard top 10 songs from a given date
# import requests

# url = "https://billboard-api2.p.rapidapi.com/hot-100"

# querystring = {"date":"2023-03-01","range":"1-10"}

# headers = {
#     # i know this is bad practice, but this is a free API key capped at 30 reqs/month
# 	"X-RapidAPI-Key": "79daae12cdmsh6e4351700c61e60p18544ejsnb1b1f2f8b40d",
# 	"X-RapidAPI-Host": "billboard-api2.p.rapidapi.com"
# }

# response = requests.request("GET", url, headers=headers, params=querystring)

top_10 = response.json()

def create_song_list(json_data):
    content = json_data['content']
    song_list = [f"{song['title']} by {song['artist']}" for song in content.values()]
    return song_list

song_list = create_song_list(top_10)

chicken_df['favorite_song'] = [random.choice(song_list) for i in range(len(chicken_df))]

chicken_df


,egg_id,generation,gender,incubation_hall_location_id,name,feather_color,favorite_song
0,90327043,1,Rooster,NaN,Mark Hernandez,white,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice"
1,25692215,1,Rooster,NaN,Victor Wells,gold,Anti-Hero by Taylor Swift
2,75991113,1,Rooster,NaN,Jeffrey Walls,gold,Kill Bill by SZA
3,35881049,1,Rooster,NaN,Ryan Green,brown,Anti-Hero by Taylor Swift
4,54519912,1,Rooster,NaN,Joseph Anderson,black,Unholy by Sam Smith & Kim Petras
...,...,...,...,...,...,...,...
995,48175098,3,Hen,8.0,Michelle Chandler,black,Die For You by The Weeknd
996,93218811,3,Hen,18.0,Mary Pope,gray,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice"
997,18358269,3,Hen,7.0,Rachel Berry,red,"Creepin' by Metro Boomin, The Weeknd & 21 Savage"
998,57497598,3,Hen,2.0,Mary George,black,Cuff It by Beyonce


In [46]:
# function to get parent hens and roosters for each generation
def get_parents(chicken_df, current_generation):
    parents_df = chicken_df[chicken_df['generation'] == current_generation - 1]
    hens = parents_df[parents_df['gender'] == 'Hen']['egg_id'].tolist()
    roosters = parents_df[parents_df['gender'] == 'Rooster']['egg_id'].tolist()
    return hens, roosters

# function to assign parent IDs to each chicken
def assign_parents(chicken_df, generation, hens, roosters):
    chicken_df.loc[chicken_df['generation'] == generation, 'parent_hen_id'] = [
        random.choice(hens) for i in range(len(chicken_df[chicken_df['generation'] == generation]))]
    chicken_df.loc[chicken_df['generation'] == generation, 'parent_rooster_id'] = [
        random.choice(roosters) for i in range(len(chicken_df[chicken_df['generation'] == generation]))]

# get parent hens and roosters for each generation
gen2_hen_parents, gen2_rooster_parents = get_parents(chicken_df, 2)
gen3_hen_parents, gen3_rooster_parents = get_parents(chicken_df, 3)

# assign parent IDs for chickens in generation 2 and 3
assign_parents(chicken_df, 2, gen2_hen_parents, gen2_rooster_parents)
assign_parents(chicken_df, 3, gen3_hen_parents, gen3_rooster_parents)

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name,feather_color,favorite_song,parent_hen_id,parent_rooster_id
0,90327043,1,Rooster,NaN,Mark Hernandez,white,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",NaN,NaN
1,25692215,1,Rooster,NaN,Victor Wells,gold,Anti-Hero by Taylor Swift,NaN,NaN
2,75991113,1,Rooster,NaN,Jeffrey Walls,gold,Kill Bill by SZA,NaN,NaN
3,35881049,1,Rooster,NaN,Ryan Green,brown,Anti-Hero by Taylor Swift,NaN,NaN
4,54519912,1,Rooster,NaN,Joseph Anderson,black,Unholy by Sam Smith & Kim Petras,NaN,NaN
...,...,...,...,...,...,...,...,...,...
995,48175098,3,Hen,8.0,Michelle Chandler,black,Die For You by The Weeknd,61654091.0,57107278.0
996,93218811,3,Hen,18.0,Mary Pope,gray,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",75396887.0,56403991.0
997,18358269,3,Hen,7.0,Rachel Berry,red,"Creepin' by Metro Boomin, The Weeknd & 21 Savage",27008865.0,84228589.0
998,57497598,3,Hen,2.0,Mary George,black,Cuff It by Beyonce,31880171.0,62843058.0


In [47]:
# function to find first cousins for a given chicken
def find_cousins(chicken_id, df):
    chicken_df = df.loc[df['egg_id'] == chicken_id]
    if chicken_df.empty:
        return []

    chicken_df = chicken_df.iloc[0]
    parent_hen = chicken_df['parent_hen_id']
    parent_rooster = chicken_df['parent_rooster_id']

    parent_hen_df = df.loc[df['egg_id'] == parent_hen]
    if parent_hen_df.empty:
        return []

    parent_hen_df = parent_hen_df.iloc[0]
    parent_hen_parent_hen = parent_hen_df['parent_hen_id']  # grandmother hen side
    parent_hen_parent_rooster = parent_hen_df['parent_rooster_id']  # grandfather hen side

    parent_hen_siblings = df.loc[
        (df['parent_hen_id'] == parent_hen_parent_hen)
        & (df['parent_rooster_id'] == parent_hen_parent_rooster)
        & (df['egg_id'] != parent_hen)
    ]

    cousins_parent_hen_side = df.loc[
        (df['parent_hen_id'].isin(parent_hen_siblings['egg_id']))
        | (df['parent_rooster_id'].isin(parent_hen_siblings['egg_id']))
    ]

    parent_rooster_df = df.loc[df['egg_id'] == parent_rooster]
    if parent_rooster_df.empty:
        return []

    parent_rooster_df = parent_rooster_df.iloc[0]
    parent_rooster_parent_hen = parent_rooster_df['parent_hen_id']  # grandmother rooster side
    parent_rooster_parent_rooster = parent_rooster_df['parent_rooster_id']  # grandfather rooster side

    parent_rooster_siblings = df.loc[
        (df['parent_hen_id'] == parent_rooster_parent_hen)
        & (df['parent_rooster_id'] == parent_rooster_parent_rooster)
        & (df['egg_id'] != parent_rooster)
    ]

    cousins_parent_rooster_side = df.loc[
        (df['parent_hen_id'].isin(parent_rooster_siblings['egg_id']))
        | (df['parent_rooster_id'].isin(parent_rooster_siblings['egg_id']))
    ]

    all_cousins = pd.concat([cousins_parent_rooster_side, cousins_parent_hen_side]).drop_duplicates()['egg_id'].to_list()

    return all_cousins

# only finding cousins for chicken in gen 3, since we don't know who grandparents are of chickens in gen 2
chicken_df['cousins'] = chicken_df.apply(lambda row: find_cousins(row['egg_id'], chicken_df) if row['generation'] == 3 else [], axis=1)

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name,feather_color,favorite_song,parent_hen_id,parent_rooster_id,cousins
0,90327043,1,Rooster,NaN,Mark Hernandez,white,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",NaN,NaN,[]
1,25692215,1,Rooster,NaN,Victor Wells,gold,Anti-Hero by Taylor Swift,NaN,NaN,[]
2,75991113,1,Rooster,NaN,Jeffrey Walls,gold,Kill Bill by SZA,NaN,NaN,[]
3,35881049,1,Rooster,NaN,Ryan Green,brown,Anti-Hero by Taylor Swift,NaN,NaN,[]
4,54519912,1,Rooster,NaN,Joseph Anderson,black,Unholy by Sam Smith & Kim Petras,NaN,NaN,[]
...,...,...,...,...,...,...,...,...,...,...
995,48175098,3,Hen,8.0,Michelle Chandler,black,Die For You by The Weeknd,61654091.0,57107278.0,[]
996,93218811,3,Hen,18.0,Mary Pope,gray,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",75396887.0,56403991.0,[]
997,18358269,3,Hen,7.0,Rachel Berry,red,"Creepin' by Metro Boomin, The Weeknd & 21 Savage",27008865.0,84228589.0,[]
998,57497598,3,Hen,2.0,Mary George,black,Cuff It by Beyonce,31880171.0,62843058.0,[]


In [48]:
# creating a 'random cousin' column for use in name tags

chicken_df['random_cousin'] = chicken_df['cousins'].apply(lambda x: random.choice(x) if x else None)

chicken_df

,egg_id,generation,gender,incubation_hall_location_id,name,feather_color,favorite_song,parent_hen_id,parent_rooster_id,cousins,random_cousin
0,90327043,1,Rooster,NaN,Mark Hernandez,white,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",NaN,NaN,[],NaN
1,25692215,1,Rooster,NaN,Victor Wells,gold,Anti-Hero by Taylor Swift,NaN,NaN,[],NaN
2,75991113,1,Rooster,NaN,Jeffrey Walls,gold,Kill Bill by SZA,NaN,NaN,[],NaN
3,35881049,1,Rooster,NaN,Ryan Green,brown,Anti-Hero by Taylor Swift,NaN,NaN,[],NaN
4,54519912,1,Rooster,NaN,Joseph Anderson,black,Unholy by Sam Smith & Kim Petras,NaN,NaN,[],NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,48175098,3,Hen,8.0,Michelle Chandler,black,Die For You by The Weeknd,61654091.0,57107278.0,[],NaN
996,93218811,3,Hen,18.0,Mary Pope,gray,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",75396887.0,56403991.0,[],NaN
997,18358269,3,Hen,7.0,Rachel Berry,red,"Creepin' by Metro Boomin, The Weeknd & 21 Savage",27008865.0,84228589.0,[],NaN
998,57497598,3,Hen,2.0,Mary George,black,Cuff It by Beyonce,31880171.0,62843058.0,[],NaN


## 3. Generate the name tags table

In [52]:
duckdb.query("""
        select
            base.name,
            base.favorite_song,
            p_hen.name as parent_hen_name,
            p_hen.incubation_hall_location_id as parent_hen_location,
            p_roo.name as parent_rooster_name,
            p_roo.incubation_hall_location_id as parent_rooster_location,
            gp_hen_hen_side.name as maternal_grandma_name,
            gp_roo_hen_side.name as maternal_grandpa_name,
            gp_hen_roo_side.name as paternal_grandma_name,        
            gp_roo_roo_side.name as paternal_grandpa_name,
            cousins.name as cousin
        from chicken_df base
        left join chicken_df p_hen
            on base.parent_hen_id = p_hen.egg_id
        left join chicken_df p_roo
            on base.parent_rooster_id = p_roo.egg_id
        left join chicken_df gp_hen_hen_side
            on p_hen.parent_hen_id = gp_hen_hen_side.egg_id    
        left join chicken_df gp_roo_hen_side
            on p_hen.parent_rooster_id = gp_roo_hen_side.egg_id        
        left join chicken_df gp_roo_roo_side
            on p_roo.parent_rooster_id = gp_roo_roo_side.egg_id  
        left join chicken_df gp_hen_roo_side
            on p_roo.parent_hen_id = gp_hen_roo_side.egg_id 
        left join chicken_df cousins
            on base.random_cousin = cousins.egg_id             
""").to_df()

,name,favorite_song,parent_hen_name,parent_hen_location,parent_rooster_name,parent_rooster_location,maternal_grandma_name,maternal_grandpa_name,paternal_grandma_name,paternal_grandpa_name,cousin
0,David Andrade,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",Suzanne Gregory,NaN,Jonathan Peck,NaN,NaN,NaN,NaN,NaN,NaN
1,Shane Wilson,Kill Bill by SZA,Ashley Nelson,NaN,Eric Anderson,NaN,NaN,NaN,NaN,NaN,NaN
2,Derek Fuller,Unholy by Sam Smith & Kim Petras,Mary Lowe,NaN,John Acevedo,NaN,NaN,NaN,NaN,NaN,NaN
3,Jeffrey Fischer,Unholy by Sam Smith & Kim Petras,Mary Lowe,NaN,Joseph Anderson,NaN,NaN,NaN,NaN,NaN,NaN
4,Luis Lopez,Die For You by The Weeknd,Megan Wood,NaN,Eric Mccoy,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,Cameron Conner,"Boy's A Liar, Pt. 2 by PinkPantheress & Ice Spice",Amanda Salazar,20.0,Thomas Medina,19.0,Kristin Mcfarland,Alex Rodriguez,Brittany Sanchez,Louis Martinez,Jason Gonzales
996,Erin Campbell,Unholy by Sam Smith & Kim Petras,Andrea Perez,3.0,Matthew Erickson,20.0,Suzanne Gregory,Mitchell Shelton,Angelica Rodriguez,David Jenkins,Christopher Taylor
997,Willie Williams,Unholy by Sam Smith & Kim Petras,Julia Grimes,24.0,John Stewart,1.0,Susan Green,Louis Martinez,Brittany Sanchez,Joshua Vargas,Paul Rivera
998,Laura Jenkins,Anti-Hero by Taylor Swift,Susan Wolfe,20.0,John Stewart,1.0,Savannah Henson,Nicholas Banks,Brittany Sanchez,Joshua Vargas,Christopher Sanders
